# SHAP

SHAP (SHapley Additive exPlanations) is a method used to explain the output of machine learning models.  
SHAP aims to explain how an input affects the output of the model, by showing the impact of each input feature on the output.  
When reading the SHAP values, you will see for each input feature how much it positively or negatively pushed the output to the answer we got, compared to the average base value of the dataset.

You can read more here: https://trustyai-explainability.github.io/trustyai-site/main/local-explainers.html

In [ ]:
!pip -q install "numpy==1.26.4" "tensorflow==2.18.0"

In [ ]:
import pickle
import pandas as pd
import numpy as np
import keras

In [3]:
OUTPUT_FEATURE = "price"

In [4]:
import warnings

# Ignore UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)

Let's start by loading some artifacts.  
We will need:
- The ONNX model
- Our pre-and-post processing artifacts
    - scaler.pkl
    - label_encoder.pkl
- Some data
    - The training inputs, these will be used to get an average input for our dataset
    - The test data, these will be used to get a point we want to analyze

In [ ]:
keras_model = keras.saving.load_model("artifacts/model.keras")

with open('artifacts/feature_scaler.pkl', 'rb') as handle:
    feature_scaler = pickle.load(handle)
with open('artifacts/label_scaler.pkl', 'rb') as handle:
    label_scaler = pickle.load(handle)

test_data = pd.read_parquet('artifacts/X_test.parquet')
train_data = pd.read_parquet('artifacts/X_train.parquet')

In [ ]:
train_data["price"].dtype == "float64"

In [ ]:
strange_prediction = test_data.loc[[532]].drop(OUTPUT_FEATURE, axis=1)
strange_prediction

TrustyAI SHAP explainer requires our model to have a pandas dataframe as an input, and numpy or pandas output, so we wrap our model in a pred() function that makes sure the input and output are converted properly. 

In [8]:
def pred(x):
    prediction = keras_model.predict(x)
    return pd.DataFrame(prediction, columns=[OUTPUT_FEATURE])

In [9]:
from trustyai.model import Model
trustyai_model = Model(pred, output_names=[OUTPUT_FEATURE])

In [ ]:
prediction = trustyai_model(strange_prediction)
prediction

In [11]:
for key in train_data.columns:
    test_data[[key]] = test_data[[key]].astype("float32")
    train_data[[key]] = train_data[[key]].astype("float32")
    try:
        strange_prediction[[key]] = strange_prediction[[key]].astype("float32")
    except:
        pass

We arbitrarily choose the first data point (song) in our test data to be the data we want to test.  
In practice, you might choose the data point that you predict the worst on, or a data point that gave an unexpected answer.  
We also look at how our data point looks when normalized (after going through pre-processing). This is how it will look like going into the model

Let's try to use our TrustyAI Model to predict the output of our data point we want to explain with SHAP.

And with everything set up, we can create a SHAP explainer and let it analyze our data point!  
You can also note that we add 100 data points from our training dataset to the SHAPExplainer, this is used to calculate the average base values of our dataset. With this, we can see how much our interesting datapoint contributes to the prediction compared to what a "standard" value would.

In [15]:
from trustyai.explainers import SHAPExplainer
explainer = SHAPExplainer(background=train_data[:100].drop(OUTPUT_FEATURE, axis=1))

In [ ]:
explanations = explainer.explain(inputs=strange_prediction,
                                 outputs=prediction,
                                 model=trustyai_model)

With our SHAP Explainer ready we can start looking at the results.

Let's choose a specific output country which we want to know how it got affected by the input values.  
CH is the country that we are supposed to get as the popular country for this input, so it's especially interesting to see the input's effect on that output.  
That being said, feel free to try with a few other countries and see what happens.  

First, we will get a table of values.  
Here we can see the **Mean Background Value** - this is the average base value we were talking about before.  
We can also see our **Value**, which is the normalized data point that we sent into the explainer. Red values are lower than the average value and green values are higher.  
Finally, we have the **SHAP Value**. These indicate how much that input feature had an effect on the output. Red indicates a negative contribution to the prediction while green a positive contribution. The larger the value, the larger the contribution.

In [ ]:
explanations.as_html()[OUTPUT_FEATURE]

We can also visualize it as a candlestick plot, seeing how the different input features build up to the output value.

In [ ]:
from trustyai.visualizations.shap import SHAPViz
SHAPViz()._matplotlib_plot(explanations=explanations, output_name=OUTPUT_FEATURE)